In [ ]:
! pip install -q tensorflow_datasets
! pip install tensorflow_text

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
import gdown
# Download the file
gdown.download('https://drive.google.com/uc?id=1zSU_1hHDtQrs4a0MoveC8gsfaYuAl81D', 'AR_EN_sentences_t.txt', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1zSU_1hHDtQrs4a0MoveC8gsfaYuAl81D
To: a:\Abdelmageed\AI with Transformers (Language version)\Mastering AI with Transformers and LLM (NLP course)\content\chapter 7 - Transformer from scratch\AR_EN_sentences_t.txt
100%|██████████| 113k/113k [00:00<00:00, 1.05MB/s]


In [ ]:
# Open the input file
with open('AR_EN_sentences_t.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Open output files for Arabic and English sentences
with open('arabic_sentences.txt', 'w', encoding='utf-8') as arabic_file, open('english_sentences.txt', 'w', encoding='utf-8') as english_file:
    for line in lines:
        # Split the line by tab
        parts = line.strip().split('\t')
        if len(parts) == 2:
            arabic_sentence, english_sentence = parts
            # Write Arabic sentence to the Arabic file
            arabic_file.write(arabic_sentence + '\n')
            # Write English sentence to the English file
            english_file.write(english_sentence + '\n')


In [ ]:
import tensorflow_datasets as tfds

class CustomTranslator(tfds.core.GeneratorBasedBuilder):
    VERSION = tfds.core.Version('2.0.0')

    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            features=tfds.features.FeaturesDict({
                'ar': tfds.features.Text(),
                'en': tfds.features.Text(),
            }),
            supervised_keys=('ar', 'en'),
            homepage='Your dataset homepage URL',
        )

    def _split_generators(self, dl_manager):
        # Download Arabic and English text files
        path_ar = dl_manager.download('/content/arabic_sentences.txt')
        path_en = dl_manager.download('/content/english_sentences.txt')

        with open(path_ar, encoding='utf-8') as f_ar, open(path_en, encoding='utf-8') as f_en:
            lines_ar = f_ar.readlines()
            lines_en = f_en.readlines()

            assert len(lines_ar) == len(lines_en), "The number of lines in Arabic and English files should match"

            num_examples = len(lines_ar)
            train_end_idx = num_examples // 5
            train_examples = self._generate_examples(lines_ar[:-train_end_idx], lines_en[:-train_end_idx])
            val_examples = self._generate_examples(lines_ar[-train_end_idx:], lines_en[-train_end_idx:])

            return {
                'train': train_examples,
                'validation': val_examples,
            }

    def _generate_examples(self, lines_ar, lines_en):
        for idx, (ar, en) in enumerate(zip(lines_ar, lines_en)):
            yield idx, {'ar': ar.strip(), 'en': en.strip()}

# Save the dataset to a specific directory
builder = CustomTranslator(data_dir='Arabic_english_translation_dataset2')
builder.download_and_prepare()
builder.as_dataset(split=['train', 'validation'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

Shuffling Arabic_english_translation_dataset2/custom_translator/2.0.0.incompleteSJ1KHM/custom_translator-train…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling Arabic_english_translation_dataset2/custom_translator/2.0.0.incompleteSJ1KHM/custom_translator-valid…

Dataset custom_translator downloaded and prepared to Arabic_english_translation_dataset2/custom_translator/2.0.0. Subsequent calls will reuse this data.


[<_PrefetchDataset element_spec={'ar': TensorSpec(shape=(), dtype=tf.string, name=None), 'en': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 <_PrefetchDataset element_spec={'ar': TensorSpec(shape=(), dtype=tf.string, name=None), 'en': TensorSpec(shape=(), dtype=tf.string, name=None)}>]

In [ ]:
import tensorflow_datasets as tfds

# Load the dataset
custom_dataset = tfds.load('custom_translator', data_dir='Arabic_english_translation_dataset2')

# Access train and validation sets
train_dataset = custom_dataset['train']
val_dataset = custom_dataset['validation']


In [ ]:
custom_dataset

{'train': <_PrefetchDataset element_spec={'ar': TensorSpec(shape=(), dtype=tf.string, name=None), 'en': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'validation': <_PrefetchDataset element_spec={'ar': TensorSpec(shape=(), dtype=tf.string, name=None), 'en': TensorSpec(shape=(), dtype=tf.string, name=None)}>}

In [ ]:
# Print an example from the dataset
for example in train_dataset.take(20):
    ar_text = example['ar'].numpy().decode('utf-8')
    en_text = example['en'].numpy().decode('utf-8')
    print({'ar': ar_text, 'en': en_text})

{'ar': 'هاجم الكلب الولد الصغير', 'en': 'The dog attacked the little boy.'}
{'ar': 'يبتغون الى ربهم الوسيله المعبودون المدعوون يخلصون لله ويوحدون الله', 'en': 'They seek a means to their Lord, the worshiped, the invoked, they devote themselves to Allah and unify Allah.'}
{'ar': 'صحيح المكان مزدحم', 'en': 'The place is crowded indeed'}
{'ar': 'فيها الراحه والامان', 'en': 'It contains comfort and security.'}
{'ar': 'وقدمنا الى ما عملوا من عمل', 'en': 'We have advanced to what they have done of work'}
{'ar': 'ونختار تصدير', 'en': 'We choose export.'}
{'ar': 'يساعدها على تقديم مقترحات وحلول', 'en': 'Helps her in providing suggestions and solutions.'}
{'ar': 'بتنبع منها الاداره كوظائف و مهام', 'en': 'Management derives from it functions and tasks.'}
{'ar': 'على اهلي', 'en': 'On my family.'}
{'ar': 'وحاول دفاعه الحصول على حكم مخفف مستندا على تاريخه المرضي وايداعه مستشفى الامراض النفسيه', 'en': 'His defense tried to secure a reduced sentence based on his satisfactory history and his admission

In [ ]:
# Print an example from the dataset
for example in val_dataset.take(20):
    ar_text = example['ar'].numpy().decode('utf-8')
    en_text = example['en'].numpy().decode('utf-8')
    print({'ar': ar_text, 'en': en_text})

{'ar': 'راح يجيك على حبه حبه بس اهم شيء تشوف تغير', 'en': 'He will come to you slowly but the most important thing is to see change.'}
{'ar': 'ماذا يحدث اذا احتاج عملاؤك في اسواق اخرى ان يعيدوا او يستبدلوا ما اشتروه', 'en': 'What happens if your customers in other markets need to return or replace what they have purchased?'}
{'ar': 'حني بالشحم ايدك عملك يصبح وليدك', 'en': 'Your work becomes your offspring when you grease your'}
{'ar': 'فلربما مثلا يتم محاصره عثمان ولكن هل يتكمن عثمان من فتح القلعه', 'en': 'Perhaps, for example, Uthman is besieged, but can Uthman manage to open the castle?'}
{'ar': 'اما في العقائد التقليد فمرفوض', 'en': 'As for blind beliefs, it is rejected.'}
{'ar': 'يعني فيه تفسير', 'en': 'It means there is interpretation'}
{'ar': 'ابني بيتي الجميل بالعزم والاصرار', 'en': 'My son, my beautiful house with determination and perseverance.'}
{'ar': 'اريد حلوه', 'en': 'I want dessert.'}
{'ar': 'بصيت وسط المعمعه شوفت سمكه ومدلعه', 'en': 'I looked in the midst of the turmoil

In [ ]:
# Get the length of train and validation datasets
train_length = tf.data.experimental.cardinality(train_dataset).numpy()
val_length = tf.data.experimental.cardinality(val_dataset).numpy()

print(f"Train dataset length: {train_length}")
print(f"Validation dataset length: {val_length}")

Train dataset length: 608721
Validation dataset length: 152180


In [ ]:
import tensorflow_datasets as tfds

# Load the dataset
custom_dataset = tfds.load('custom_translator', data_dir='Arabic_english_translation_dataset')

# Access train dataset
train_dataset = custom_dataset['train']

# Function to extract Arabic sentences
def get_arabic(example):
    return example['ar']

# Function to extract English sentences
def get_english(example):
    return example['en']

# Apply the functions to get Arabic and English sentences separately
train_arabic = train_dataset.map(lambda x: get_arabic(x))
train_english = train_dataset.map(lambda x: get_english(x))

# Batching and prefetching
train_arabic_batched = train_arabic.batch(1000).prefetch(2)
train_english_batched = train_english.batch(1000).prefetch(2)


In [ ]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

# we will consider some of special token such as padding , unknown vocabulary,
#start and end of the sequence in our argument dictionary , These tokens are important for BERT to handle various tasks like padding sequences and indicating the start and end of sentences.
special_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

#and then we define bert_tokenizer_params dictionary that contains parameters for the BERT tokenizer. In this case, lower_case is set to True, which means that the tokenizer should convert all text to lowercase before tokenizing. This is a common preprocessing step used in NLP tasks.
bert_tokenizer_params=dict(lower_case=True)
# we will add these two variables to the arguments beside The target vocabulary size
bert_vocab_args = dict(
    	reserved_tokens = special_tokens,
    	bert_tokenizer_params = bert_tokenizer_params,
    	vocab_size = 8000,
      # also we should consider arguments related to wordpiece tokenization. However,
      #we left it empty, there no specific parameters we need to add .
      learn_params={},
)

# now we can generate the vocabularies for Russian and English
arabic_vocab = bert_vocab.bert_vocab_from_dataset( train_arabic.batch(1000).prefetch(2),
                                               **bert_vocab_args )

english_vocab = bert_vocab.bert_vocab_from_dataset( train_english.batch(1000).prefetch(2),
                                              **bert_vocab_args )


In [ ]:
print("arabic ")
print(arabic_vocab[:10])
print(arabic_vocab[100:110])
print(arabic_vocab[-5:])
print("English")
print(english_vocab[:10])
print(english_vocab[100:110])
print(english_vocab[-5:])

arabic 
['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '%', "'", '(', ')']
['٧', '٨', '٩', '٪', '٫', 'ٺ', 'چ', 'ڤ', 'ک', 'گ']
['##ی', '##ە', '##–', '##’', '##→']
English
['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '$', '%', '&']
['ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه']
['##’', '##“', '##”', '##→', '##⟶']


In [ ]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

write_vocab_file('arabic_vocab.txt', arabic_vocab)
write_vocab_file('english_vocab.txt', english_vocab)

In [ ]:
import tensorflow_text as text
import re
import pathlib
arabic_tokenizer = text.BertTokenizer('arabic_vocab.txt', **bert_tokenizer_params)
english_tokenizer = text.BertTokenizer('english_vocab.txt', **bert_tokenizer_params)

special_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

In [ ]:
def cleanup_text(special_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in special_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)
  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)
  return result

In [ ]:
special_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

# we will store the indices of the [START] and [END] tokens in the reserved_tokens list, respectively.
#They are computed using TensorFlow's argmax function in combination with boolean comparison.

START = tf.argmax(tf.constant(special_tokens) == "[START]") # <tf.Tensor: shape=(), dtype=int64, numpy=3>
END = tf.argmax(tf.constant(special_tokens) == "[END]") # <tf.Tensor: shape=(), dtype=int64, numpy=3>
#We can use add_start_end function that takes a ragged tensor as input and adds [START] and [END] tokens to each sequence within the tensor
def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [ ]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._special_tokens = special_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:

    # Include a tokenize signature for a batch of strings.
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._special_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._special_tokens)

In [ ]:
tokenizers = tf.Module()
tokenizers.arabic = CustomTokenizer(special_tokens, 'arabic_vocab.txt')
tokenizers.english = CustomTokenizer(special_tokens, 'english_vocab.txt')


In [ ]:
model_name = 'arabic_english_tokenizer'
tf.saved_model.save(tokenizers, model_name)


In [ ]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.english.get_vocab_size().numpy()

7860

In [ ]:
reloaded_tokenizers.arabic.get_vocab_size().numpy()

7898

In [ ]:
reloaded_tokenizers = tf.saved_model.load(model_name)
tokens = reloaded_tokenizers.arabic.tokenize([' سبحان الله وبحمده سبحان الله العظيم أهلا وسهلا  كيف الحال'])
tokens.numpy()


array([[   2, 1693,  122,   90, 1854, 6626, 1693,  122,  843, 1189, 4310,
         208, 1696,    3]])

In [ ]:
round_trip = reloaded_tokenizers.arabic.detokenize(tokens)
print(round_trip.numpy()[0].decode('utf-8'))


سبحان الله وبحمده سبحان الله العظيم اهلا وسهلا كيف الحال
